In [1]:
import os
print(os.environ.get("KAGGLE_USERNAME"))
print(os.environ.get("KAGGLE_KEY"))


EZE GOSPEL
KGAT_48e9e9343c9f87722cdf1b867675ded5


In [2]:
!kaggle datasets list -s "supply chain"


HTTPSConnectionPool(host='api.kaggle.com', port=443): Max retries exceeded with url: /v1/datasets.DatasetApiService/ListDatasets (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001E98E86BF90>: Failed to resolve 'api.kaggle.com' ([Errno 11001] getaddrinfo failed)"))


In [3]:
# Step 1: Download the dataset from Kaggle
!kaggle datasets download -d harshsingh2209/supply-chain-analysis

# Step 2: Unzip the dataset into a folder
import zipfile

with zipfile.ZipFile("supply-chain-analysis.zip", 'r') as zip_ref:
    zip_ref.extractall("supply_chain_data")

# Step 3: Load the dataset into Pandas
import pandas as pd

# Replace with the actual CSV filename inside the zip (often 'supply_chain_data.csv')
df = pd.read_csv("supply_chain_data/supply_chain_data.csv")

# Step 4: Preview the first few rows
print(df.head())


HTTPSConnectionPool(host='api.kaggle.com', port=443): Max retries exceeded with url: /v1/datasets.DatasetApiService/GetDatasetMetadata (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001DBB3E3BFD0>: Failed to resolve 'api.kaggle.com' ([Errno 11001] getaddrinfo failed)"))
  Product type   SKU      Price  Availability  Number of products sold  \
0     haircare  SKU0  69.808006            55                      802   
1     skincare  SKU1  14.843523            95                      736   
2     haircare  SKU2  11.319683            34                        8   
3     skincare  SKU3  61.163343            68                       83   
4     skincare  SKU4   4.805496            26                      871   

   Revenue generated Customer demographics  Stock levels  Lead times  \
0        8661.996792            Non-binary            58           7   
1        7460.900065                Female            53          30   
2        9577.749626            

In [4]:
import pandas as pd
from sqlalchemy import create_engine

# Load your dataset
df = pd.read_csv("supply_chain_data/supply_chain_data.csv")

# Create SQLite engine
engine = create_engine("sqlite:///supply_chain.db", echo=False)

# Save dataframe to SQL table
df.to_sql("supply_chain", con=engine, if_exists="replace", index=False)


100

In [5]:
import sqlite3

conn = sqlite3.connect("supply_chain.db")
cursor = conn.cursor()

# Example 1: Top 5 products by revenue
query1 = """
SELECT SKU, SUM("Revenue generated") AS total_revenue
FROM supply_chain
GROUP BY SKU
ORDER BY total_revenue DESC
LIMIT 5;
"""
for row in cursor.execute(query1):
    print(row)

# Example 2: Average lead time by product type
query2 = """
SELECT "Product type", AVG("Lead times") AS avg_lead_time
FROM supply_chain
GROUP BY "Product type";
"""
for row in cursor.execute(query2):
    print(row)


('SKU51', 9866.465457979695)
('SKU38', 9692.318040218432)
('SKU31', 9655.135102719398)
('SKU90', 9592.633570280312)
('SKU2', 9577.74962586873)
('cosmetics', 15.384615384615385)
('haircare', 15.529411764705882)
('skincare', 16.7)


In [6]:
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect("supply_chain.db")
cursor = conn.cursor()

# Run SQL query
query = """
SELECT "Customer demographics", SUM("Revenue generated") AS total_revenue
FROM supply_chain
GROUP BY "Customer demographics"
ORDER BY total_revenue DESC;
"""

for row in cursor.execute(query):
    print(row)


('Unknown', 173090.13383697468)
('Female', 161514.48912163064)
('Male', 126634.39425979402)
('Non-binary', 116365.80151960932)


In [7]:
%load_ext sql
%sql sqlite:///supply_chain.db


Connecting to 'sqlite:///supply_chain.db'

In [8]:
%%sql
SELECT * FROM supply_chain
LIMIT 5;


Running query in 'sqlite:///supply_chain.db'

Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,Shipping times,Shipping carriers,Shipping costs,Supplier name,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
haircare,SKU0,69.80800554211577,55,802,8661.996792392383,Non-binary,58,7,96,4,Carrier B,2.956572139430807,Supplier 3,Mumbai,29,215,29,46.27987924050832,Pending,0.2264103608499251,Road,Route B,187.75207545920392
skincare,SKU1,14.84352327508434,95,736,7460.900065445849,Female,53,30,37,2,Carrier A,9.71657477143131,Supplier 3,Mumbai,23,517,30,33.61676895373,Pending,4.854068026388706,Road,Route B,503.0655791496692
haircare,SKU2,11.319683293090566,34,8,9577.74962586873,Unknown,1,10,88,2,Carrier B,8.054479261732155,Supplier 1,Mumbai,12,971,27,30.688019348284204,Pending,4.580592619199229,Air,Route C,141.92028177151906
skincare,SKU3,61.163343016437736,68,83,7766.836425685233,Non-binary,23,13,59,6,Carrier C,1.7295685635434288,Supplier 5,Kolkata,24,937,18,35.62474139712503,Fail,4.74664862064775,Rail,Route A,254.77615921928663
skincare,SKU4,4.805496036345893,26,871,2686.505151567447,Non-binary,5,3,56,8,Carrier A,3.890547915870672,Supplier 1,Delhi,5,414,3,92.06516059871284,Fail,3.145579522833002,Air,Route A,923.4406317119222


In [9]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Float, String

# Load dataset
df = pd.read_csv("supply_chain_data/supply_chain_data.csv")

# Explicit conversions
df["Availability"] = pd.to_numeric(df["Availability"], errors="coerce").astype("Int64")
df["Number of products sold"] = pd.to_numeric(df["Number of products sold"], errors="coerce").astype("Int64")
df["Stock levels"] = pd.to_numeric(df["Stock levels"], errors="coerce").astype("Int64")
df["Lead times"] = pd.to_numeric(df["Lead times"], errors="coerce").astype("Int64")
df["Order quantities"] = pd.to_numeric(df["Order quantities"], errors="coerce").astype("Int64")
df["Shipping times"] = pd.to_numeric(df["Shipping times"], errors="coerce").astype("Int64")
df["Production volumes"] = pd.to_numeric(df["Production volumes"], errors="coerce").astype("Int64")
df["Manufacturing lead time"] = pd.to_numeric(df["Manufacturing lead time"], errors="coerce").astype("Int64")
df["Lead time"] = pd.to_numeric(df["Lead time"], errors="coerce").astype("Int64")

# Floats
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")
df["Revenue generated"] = pd.to_numeric(df["Revenue generated"], errors="coerce")
df["Shipping costs"] = pd.to_numeric(df["Shipping costs"], errors="coerce")
df["Manufacturing costs"] = pd.to_numeric(df["Manufacturing costs"], errors="coerce")
df["Defect rates"] = pd.to_numeric(df["Defect rates"], errors="coerce")
df["Costs"] = pd.to_numeric(df["Costs"], errors="coerce")

# Strings
text_columns = ["Product type","SKU","Customer demographics","Shipping carriers",
                "Supplier name","Location","Inspection results","Transportation modes","Routes"]
for col in text_columns:
    df[col] = df[col].astype(str)


In [10]:
dtype_map = {
    "Product type": String(),
    "SKU": String(),
    "Price": Float(),
    "Availability": Integer(),
    "Number of products sold": Integer(),
    "Revenue generated": Float(),
    "Customer demographics": String(),
    "Stock levels": Integer(),
    "Lead times": Integer(),
    "Order quantities": Integer(),
    "Shipping times": Integer(),
    "Shipping carriers": String(),
    "Shipping costs": Float(),
    "Supplier name": String(),
    "Location": String(),
    "Lead time": Integer(),
    "Production volumes": Integer(),
    "Manufacturing lead time": Integer(),
    "Manufacturing costs": Float(),
    "Inspection results": String(),
    "Defect rates": Float(),
    "Transportation modes": String(),
    "Routes": String(),
    "Costs": Float()
}

engine = create_engine(
    "sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db",
    connect_args={'timeout': 30},  # wait if locked
    echo=False
)

df.to_sql("supply_chain", con=engine, if_exists="replace", index=False, dtype=dtype_map)
print("Data saved successfully to supply_chain_clean.db")


Data saved successfully to supply_chain_clean.db


In [11]:
%%sql sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db
PRAGMA table_info(supply_chain);


Connecting and switching to connection 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

cid,name,type,notnull,dflt_value,pk
0,Product type,VARCHAR,0,None,0
1,SKU,VARCHAR,0,None,0
2,Price,FLOAT,0,None,0
3,Availability,INTEGER,0,None,0
4,Number of products sold,INTEGER,0,None,0
5,Revenue generated,FLOAT,0,None,0
6,Customer demographics,VARCHAR,0,None,0
7,Stock levels,INTEGER,0,None,0
8,Lead times,INTEGER,0,None,0
9,Order quantities,INTEGER,0,None,0


In [12]:
%%sql
SELECT COUNT(*) 
FROM supply_chain 
WHERE "Revenue generated" IS NULL;



Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

COUNT(*)
0


In [13]:
%%sql
SELECT SKU, COUNT(*) AS count_rows
FROM supply_chain
GROUP BY SKU
HAVING COUNT(*) > 1;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

SKU,count_rows


In [14]:
%%sql
PRAGMA table_info(supply_chain);


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

cid,name,type,notnull,dflt_value,pk
0,Product type,VARCHAR,0,None,0
1,SKU,VARCHAR,0,None,0
2,Price,FLOAT,0,None,0
3,Availability,INTEGER,0,None,0
4,Number of products sold,INTEGER,0,None,0
5,Revenue generated,FLOAT,0,None,0
6,Customer demographics,VARCHAR,0,None,0
7,Stock levels,INTEGER,0,None,0
8,Lead times,INTEGER,0,None,0
9,Order quantities,INTEGER,0,None,0


In [15]:
%%sql
SELECT DISTINCT "Customer demographics"
FROM supply_chain;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

Customer demographics
Non-binary
Female
Unknown
Male


In [16]:
%%sql
SELECT DISTINCT "Transportation modes"
FROM supply_chain;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

Transportation modes
Road
Air
Rail
Sea


In [17]:
%%sql
SELECT *
FROM supply_chain
WHERE "Lead times" < 0 OR "Defect rates" < 0;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,Shipping times,Shipping carriers,Shipping costs,Supplier name,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs


In [18]:
%%sql
SELECT 
    SUM(CASE WHEN "Price" IS NULL THEN 1 ELSE 0 END) AS missing_price,
    SUM(CASE WHEN "Availability" IS NULL THEN 1 ELSE 0 END) AS missing_availability,
    SUM(CASE WHEN "Revenue generated" IS NULL THEN 1 ELSE 0 END) AS missing_revenue,
    SUM(CASE WHEN "Customer demographics" IS NULL THEN 1 ELSE 0 END) AS missing_demographics
FROM supply_chain;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

missing_price,missing_availability,missing_revenue,missing_demographics
0,0,0,0


In [19]:
%%sql
SELECT *
FROM supply_chain
WHERE "Defect rates" < 0 
   OR "Lead times" < 0 
   OR "Manufacturing costs" < 0 
   OR "Price" < 0;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,Shipping times,Shipping carriers,Shipping costs,Supplier name,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs


In [20]:
%%sql
SELECT "Product type", SUM("Revenue generated") AS total_revenue
FROM supply_chain
GROUP BY "Product type"
ORDER BY total_revenue DESC;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

Product type,total_revenue
skincare,241628.16213306322
haircare,174455.3906054623
cosmetics,161521.26599948315


In [21]:
%%sql
SELECT Location, AVG("Lead times") AS avg_lead_time
FROM supply_chain
GROUP BY Location
ORDER BY avg_lead_time DESC;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

Location,avg_lead_time
Delhi,17.666666666666668
Chennai,17.15
Kolkata,15.92
Bangalore,15.333333333333334
Mumbai,14.272727272727273


In [22]:
%%sql
SELECT "Supplier name", SUM("Revenue generated") AS total_revenue
FROM supply_chain
GROUP BY "Supplier name"
ORDER BY total_revenue DESC;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

Supplier name,total_revenue
Supplier 1,157528.99503945644
Supplier 2,125467.4186053003
Supplier 5,110343.4636562869
Supplier 3,97795.97963816227
Supplier 4,86468.96179880276


In [23]:
%%sql
SELECT "Transportation modes", AVG("Defect rates") AS avg_defect
FROM supply_chain
GROUP BY "Transportation modes"
ORDER BY avg_defect DESC;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

Transportation modes,avg_defect
Road,2.620938470702914
Rail,2.3188139085148194
Sea,2.31528129391858
Air,1.8239235457132958


In [24]:
%%sql
SELECT SKU, "Manufacturing costs"
FROM supply_chain
ORDER BY "Manufacturing costs" DESC
LIMIT 5;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

SKU,Manufacturing costs
SKU7,99.46610860359912
SKU23,98.60995724270389
SKU13,97.82905011017328
SKU83,97.73059380053304
SKU15,97.12128175147431


In [25]:
%%sql
SELECT "Shipping carriers", AVG("Shipping costs") AS avg_shipping_cost
FROM supply_chain
GROUP BY "Shipping carriers"
ORDER BY avg_shipping_cost DESC;


Running query in 'sqlite:///C:/Users/HP/Downloads/supply_chain_clean.db'

Shipping carriers,avg_shipping_cost
Carrier C,5.599291618337506
Carrier A,5.554922521651271
Carrier B,5.50924696892873


In [26]:
import pandas as pd
from sqlalchemy import create_engine

# Step 1: Load dataset
df = pd.read_csv("supply_chain_data/supply_chain_data.csv")

# Step 2: Inspect current dtypes
print("Before conversion:\n", df.dtypes)

# Step 3: Convert numeric-looking columns
# Try to convert every column to numeric; if it fails, keep as text
for col in df.columns:
    # Attempt numeric conversion
    converted = pd.to_numeric(df[col], errors="coerce")
    # If at least half the values are numeric, keep conversion
    if converted.notna().sum() > len(df[col]) / 2:
        # Decide integer vs float
        if (converted.dropna() % 1 == 0).all():
            df[col] = converted.astype("Int64")   # Whole numbers
        else:
            df[col] = converted.astype(float)     # Decimals
    else:
        df[col] = df[col].astype(str)             # Keep as text

# Step 4: Inspect after conversion
print("After conversion:\n", df.dtypes)

# Step 5: Save into SQLite in Downloads
engine = create_engine("sqlite:///C:/Users/HP/Downloads/supply_chain_fixed.db", echo=False)
df.to_sql("supply_chain", con=engine, if_exists="replace", index=False)

print("Data saved successfully to supply_chain_fixed.db")


Before conversion:
 Product type                object
SKU                         object
Price                      float64
Availability                 int64
Number of products sold      int64
Revenue generated          float64
Customer demographics       object
Stock levels                 int64
Lead times                   int64
Order quantities             int64
Shipping times               int64
Shipping carriers           object
Shipping costs             float64
Supplier name               object
Location                    object
Lead time                    int64
Production volumes           int64
Manufacturing lead time      int64
Manufacturing costs        float64
Inspection results          object
Defect rates               float64
Transportation modes        object
Routes                      object
Costs                      float64
dtype: object
After conversion:
 Product type                object
SKU                         object
Price                      float64
Av

In [27]:
import sqlite3

# Connect to your database
conn = sqlite3.connect("supply_chain.db")
cursor = conn.cursor()

# Check table schema
cursor.execute("PRAGMA table_info(supply_chain);")
for row in cursor.fetchall():
    print(row)


(0, 'Product type', 'TEXT', 0, None, 0)
(1, 'SKU', 'TEXT', 0, None, 0)
(2, 'Price', 'FLOAT', 0, None, 0)
(3, 'Availability', 'BIGINT', 0, None, 0)
(4, 'Number of products sold', 'BIGINT', 0, None, 0)
(5, 'Revenue generated', 'FLOAT', 0, None, 0)
(6, 'Customer demographics', 'TEXT', 0, None, 0)
(7, 'Stock levels', 'BIGINT', 0, None, 0)
(8, 'Lead times', 'BIGINT', 0, None, 0)
(9, 'Order quantities', 'BIGINT', 0, None, 0)
(10, 'Shipping times', 'BIGINT', 0, None, 0)
(11, 'Shipping carriers', 'TEXT', 0, None, 0)
(12, 'Shipping costs', 'FLOAT', 0, None, 0)
(13, 'Supplier name', 'TEXT', 0, None, 0)
(14, 'Location', 'TEXT', 0, None, 0)
(15, 'Lead time', 'BIGINT', 0, None, 0)
(16, 'Production volumes', 'BIGINT', 0, None, 0)
(17, 'Manufacturing lead time', 'BIGINT', 0, None, 0)
(18, 'Manufacturing costs', 'FLOAT', 0, None, 0)
(19, 'Inspection results', 'TEXT', 0, None, 0)
(20, 'Defect rates', 'FLOAT', 0, None, 0)
(21, 'Transportation modes', 'TEXT', 0, None, 0)
(22, 'Routes', 'TEXT', 0, None, 0

In [28]:
import pandas as pd

df = pd.read_sql_query(
    "SELECT SKU, \"Lead time\", \"Lead times\" FROM supply_chain LIMIT 10;", conn
)
print(df)


    SKU  Lead time  Lead times
0  SKU0         29           7
1  SKU1         23          30
2  SKU2         12          10
3  SKU3         24          13
4  SKU4          5           3
5  SKU5         10          27
6  SKU6         14          15
7  SKU7         22          17
8  SKU8         13          10
9  SKU9         29          27


In [29]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM supply_chain LIMIT 5;", conn)

print(df.dtypes)   # shows data types
print(df.columns)  # lists all column names


Product type                object
SKU                         object
Price                      float64
Availability                 int64
Number of products sold      int64
Revenue generated          float64
Customer demographics       object
Stock levels                 int64
Lead times                   int64
Order quantities             int64
Shipping times               int64
Shipping carriers           object
Shipping costs             float64
Supplier name               object
Location                    object
Lead time                    int64
Production volumes           int64
Manufacturing lead time      int64
Manufacturing costs        float64
Inspection results          object
Defect rates               float64
Transportation modes        object
Routes                      object
Costs                      float64
dtype: object
Index(['Product type', 'SKU', 'Price', 'Availability',
       'Number of products sold', 'Revenue generated', 'Customer demographics',
       'Sto

In [30]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///C:/Users/HP/Downloads/supply_chain.db", connect_args={'check_same_thread': False}, echo=False)


In [31]:
import os
os.path.abspath("supply_chain.db")


'C:\\Users\\HP\\supply_chain.db'

In [32]:
%%sql sqlite:///C:/Users/HP/supply_chain.db
SELECT name FROM sqlite_master WHERE type='table';



Connecting and switching to connection 'sqlite:///C:/Users/HP/supply_chain.db'

name
supply_chain


In [33]:
import os
os.path.exists(r"C:\Users\HP\Downloads\supply_chain.db")


True